In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import h5py
import tqdm


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

import sys
sys.path.append('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation')
import anatomy as anat

In [ ]:
file_path = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss64_09Bn1000_2023-06-22_uniform-09b_n1000_r100_f40_l10'
hdf = h5py.File(file_path+'/sam_lib.hdf5', 'r')

print(hdf.keys())
sample_params = hdf['sample_params'][...]
hc_ss = hdf['hc_ss'][...]
hc_bg = hdf['hc_bg'][...]
sspar = hdf['sspar'][...]
bgpar = hdf['bgpar'][...]
fobs_gw_cents = hdf['fobs'][:]
hdf.close()

dur = 1/fobs_gw_cents[0]
fobs_gw_cents, fobs_gw_edges = holo.utils.pta_freqs(dur, num=len(fobs_gw_cents))
nsamps, nfreqs, nreals, nloudest = [*hc_ss.shape]


In [ ]:
npz = np.load(file_path+'/PS_Uniform_09B.pspace.npz', allow_pickle=True)
print(npz.files)
param_names = npz['param_names']
print(param_names)
print(npz['class_name'])
lib_name = '%s v%s, %d samples' % (npz['class_name'], npz['librarian_version'], nsamps)
print(lib_name)
# print(npz['sam_shape'])
npz.close()

# Load detstats

In [ ]:
npz = np.load('/Users/emigardiner/GWs/holodeck/output/brc_output/ss64_09Bn1000_2023-06-22_uniform-09b_n1000_r100_f40_l10/detstats/psrs40_sigma4.31e-06/detstats.npz')
print(npz.files)
# print(npz['snr_ss'].shape)
dp_ss = npz['dp_ss']
dp_bg = npz['dp_bg']
df_ss = npz['df_ss']
df_bg = npz['df_bg']
ev_ss = npz['ev_ss']
print(f"{ev_ss.shape=}, {dp_ss.shape=}")
npz.close()

In [ ]:
print(sample_params[0])

In [ ]:
# def_draw_ev(ax):
# plt.rcParams['mathtext.fontset'] = "cm"
# plt.rcParams["font.family"] = "serif"

def draw_skies_vs_bg(ax, skies_ss, dp_bg, label=None,
                     color='k', mean=True, errors=False,
                     ms=2):
    xx = dp_bg # shape (R,)
    if mean:
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    else: 
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    yerr = np.std(skies_ss, axis=-1) # shape (R,)
    if errors:
        hh = ax.errorbar(xx, yy, yerr, color=color, label=label,
                    linestyle='', capsize=3, marker='o', alpha=0.05, markersize=ms)
    else: 
        hh = ax.scatter(xx, yy, color=color, label=label,
                    linestyle='', marker='o', alpha=0.05, s=ms)
    return hh

def plot_dpss_vs_dpbg(dp_ss, dp_bg, target_params=None, norm=None, 
                      use_ev=False, errors=False, ms=2, alpha=0.05, color='tab:blue', cmap=cm.rainbow):
    # colors = cm.rainbow_r(np.linspace(0, 1, len(dsdat)))
    # cmin = 

    sslabel = 'Expected Number' if use_ev else 'Detection Probability'
    fig, ax = plot.figax(xlabel='Background Detection Probability', 
                         ylabel='$\langle$ Single Source %s $\\rangle_\mathrm{skies}$' % sslabel,
    ) 
    
    for ii in range(len(dp_ss)):
        # label = '%.2f' % params[ii][target_param]
        if target_params is not None and norm is not None:
            color = cmap(norm(target_params[ii]))
        detss = dp_ss[ii] # use_ev else ds['dp_ss']
        hh = draw_skies_vs_bg(ax, detss, dp_bg[ii], color=color, errors=errors, ms=ms) #, label=label)


    fig.tight_layout()
    return fig




# fig2 = plot_dpss_vs_dpbg(ev_ss, dp_bg, use_ev=True, errors=True)

In [ ]:
fig1 = plot_dpss_vs_dpbg(dp_ss, dp_bg, use_ev=False)
fig2 = plot_dpss_vs_dpbg(ev_ss, dp_bg, use_ev=True)

# Color by params

In [ ]:
target = 'hard_time'
def normalize_target(target, param_names=param_names, sample_params=sample_params):
    idx = list(param_names).index(target)
    target_params = sample_params[:,idx]
    mintarget = np.min(target_params)
    maxtarget = np.max(target_params)
    norm = mpl.colors.Normalize(vmin=mintarget, vmax=maxtarget)
    return target_params, norm

Connect to hard_time

In [ ]:
target_params, norm = normalize_target('hard_time')
cmap=cm.rainbow
fig = plot_dpss_vs_dpbg(ev_ss, dp_bg, use_ev=True, target_params=target_params, norm=norm, 
                        errors=False, cmap=cmap, ms=5, alpha=0.1)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), 
                    label='$\\tau_\mathrm{hard}$', shrink=0.75 )

In [ ]:
print(nsamps, nfreqs, nreals)

In [ ]:
print(ev_ss.shape)

In [ ]:
print(param_names)

In [ ]:
cmap=cm.rainbow

for tt, target in enumerate(param_names):
    target_params, norm = normalize_target(target)
    fig = plot_dpss_vs_dpbg(ev_ss, dp_bg, use_ev=True, target_params=target_params, norm=norm, 
                            errors=False, cmap=cmap, ms=5, alpha=0.1)
    cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), 
                        label=target, shrink=0.75 )
    # fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/params/ss_vs_bg_dp_lib09A_single_pta/{target}.png')

In [ ]:
fig.axes[0].set_ylim(10**-1, 4)
fig

# Number Density of Ratio vs. Parameter

## Ratio

In [ ]:
nskies = ev_ss.shape[-1]
ratio = ev_ss/dp_bg[:,:,np.newaxis]
nbins=40
ratio_edges = np.geomspace(np.min(ratio), np.max(ratio), nbins)
print(f"{ratio.shape=}")

### Ratio vs Hardening Time

In [ ]:
thard = np.repeat(sample_params[:,0], nreals*nskies).reshape(nsamps, nreals, nskies)
print(f"{thard.shape=}")
th_edges = np.geomspace(np.min(sample_params[:,0]), np.max(sample_params[:,0]), nbins)
hist, dpe, the = np.histogram2d(ratio.flatten(), thard.flatten(), bins=(ratio_edges, th_edges))

DP, TH = np.meshgrid(ratio_edges, th_edges)
fig, ax = plot.figax(xlabel='Hardening Time $\\tau_\mathrm{hard}$', ylabel='Ratio $\langle N_\mathrm{SS} \\rangle / \mathrm{DP}_\mathrm{BG}$')
ax.set_facecolor('k')
im = ax.pcolormesh(TH, DP, np.log10(hist))
cbar = plt.colorbar(im, ax=ax, label='$\log N$')

### Ratio vs GSMF Mchar0

In [ ]:
print(param_names)

In [ ]:
mchar = np.repeat(sample_params[:,2], nreals*nskies).reshape(nsamps, nreals, nskies)
print(f"{mchar.shape=}")
mc_edges = np.linspace(np.min(sample_params[:,2]), np.max(sample_params[:,2]), nbins)
hist, mce, the = np.histogram2d(ratio.flatten(), mchar.flatten(), bins=(ratio_edges, mc_edges))

DP, MC = np.meshgrid(ratio_edges, mc_edges)
fig, ax = plot.figax(xlabel='log($M_\mathrm{char,0} / \mathrm{M}_\odot)$', ylabel='Ratio $\langle N_\mathrm{SS} \\rangle / \mathrm{DP}_\mathrm{BG}$',
                     xscale='linear')
ax.set_facecolor('k')
im = ax.pcolormesh(MC, DP, np.log10(hist))
cbar = plt.colorbar(im, ax=ax, label='$\log N$')

## Background DP vs Param

In [ ]:
def plot_2D_hist(xx, yy, xlabel, ylabel, xscale, nbins=40):
   if xscale == 'linear':
      xx_edges = np.linspace(np.min(xx), np.max(xx), nbins)
   else:
      xx_edges = np.geomspace(np.min(xx), np.max(xx), nbins)
   print(f"{np.min(xx)=}, {np.max(xx)=}, {np.min(yy)=}, {np.max(yy)=}")
   yy_edges = np.geomspace(np.min(yy), np.max(yy), nbins)
   print(f"{xx.shape=}, {yy.shape=}")

   hist, temp1, temp2 = np.histogram2d(dp_bg.flatten(), thard.flatten(), bins=(yy_edges, xx_edges))

   YY, XX = np.meshgrid(yy_edges, xx_edges)
   fig, ax = plot.figax_single(xlabel=xlabel, ylabel=ylabel,
                     xscale=xscale)
   ax.set_facecolor('k')
   im = ax.pcolormesh(XX, YY, np.log10(hist))
   cbar = plt.colorbar(im, ax=ax, label='$\log N$')
   return fig, hist

### hard time

In [ ]:
xx = np.repeat(sample_params[:,0], nreals).reshape(nsamps, nreals)
yy = dp_bg
xlabel = '$\\tau_\mathrm{hard}$'
ylabel = '$\mathrm{DP}_\mathrm{BG}$'
xscale = 'linear'
fig, hist = plot_2D_hist(xx,yy,xlabel,ylabel,xscale)

### gsmf phi0

In [ ]:
plt.hist(sample_params[:,1], bins=200)

In [ ]:
xx = np.repeat(sample_params[:,1], nreals).reshape(nsamps, nreals)
yy = dp_bg
xlabel = 'GSMF $\psi_0$'
ylabel = '$\mathrm{DP}_\mathrm{BG}$'
xscale = 'linear'
fig, hist = plot_2D_hist(xx,yy,xlabel,ylabel,xscale,)

print(f"{holo.utils.stats(xx)=}, \n{holo.utils.stats(yy)=}, \n{holo.utils.stats(hist)=}")

In [ ]:
xx = np.repeat(sample_params[:,3], nreals).reshape(nsamps, nreals)
yy = dp_bg
xlabel = 'MMB $\log(M_\mathrm{amp}/\mathrm{M}_\odot)$'
ylabel = '$\mathrm{DP}_\mathrm{BG}$'
xscale = 'linear'
fig, hist = plot_2D_hist(xx,yy,xlabel,ylabel,xscale)

print(f"{holo.utils.stats(xx)=}, \n{holo.utils.stats(yy)=}, \n{holo.utils.stats(hist)=}")